In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial
import statsmodels.api as sm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree, 
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GB, 
    GradientBoostingClassifier as GC)
from ISLP.bart import BART
import sklearn.model_selection as skm
import seaborn as sns
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

In [6]:
data = pd.read_pickle("data.pkl")

In [9]:
# Define triple interactions
triple_interactions = [
    "type_1_female_public",
    "type_2_female_public",
    "type_3_female_public",
    "type_4_female_public"
]

# Define all regressors: main effects + two-way + triple interactions
X_vars = [
    "female",
    "public_facing",
    "type_1", "type_2", "type_3", "type_4",
    "female_publicOcc",
    "type_1_female", "type_2_female", "type_3_female", "type_4_female",
    "type_1_public", "type_2_public", "type_3_public", "type_4_public",
    "type_1_female_public", "type_2_female_public", "type_3_female_public", "type_4_female_public"
]

# Outcome variable
y = data["callback"]

# Prepare design matrix
X = data[X_vars]
X = sm.add_constant(X)

# Fit baseline LPM
lpm_model = sm.OLS(y, X).fit()

# Function to add stars based on p-value
def signif_stars(p):
    if p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.1:
        return '*'
    else:
        return ''

# Prepare table with only triple interactions
results = []
for t in triple_interactions:
    coef = lpm_model.params[t]
    pval = lpm_model.pvalues[t]
    stars = signif_stars(pval)
    
    results.append({
        "Treatment": t,
        "LPM Coef": f"{coef:.3f}{stars}",
        "p-value": round(pval, 3)
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Save as LaTeX
results_df.to_latex("triple_interactions_lpm_stars.tex", index=False, escape=False, float_format="%.3f")

print(results_df)


              Treatment LPM Coef  p-value
0  type_1_female_public    0.023    0.521
1  type_2_female_public    0.018    0.653
2  type_3_female_public    0.014    0.740
3  type_4_female_public   -0.016    0.703
